In [30]:
import os
import gspread
import cryptocode
import pandas as pd
import requests
from dotenv import load_dotenv
load_dotenv()

True

In [31]:
AWS_KEY = os.environ.get('AWS_KEY')
AWS_SECRET = os.environ.get('AWS_SECRET')
Gspread='https://docs.google.com/spreadsheets/d/1vhP8osxR0snaPJJaU36AbaNi6pwBfkCnuNCMNJQsdYA/edit#gid=0'

In [14]:
def get_sql(AWS_KEY, AWS_SECRET, sql_command):
    import cryptocode
    import pymysql
    import s3fs
    # 암호화된 태블로 DB 접속키 로딩
    # 암호화된 태블로 DB 접속키 로딩
    fs = s3fs.S3FileSystem(anon=False, key=AWS_KEY, secret=AWS_SECRET)
    with fs.open("s3://zigbang-mlops/models/red/red_encrypted.txt", mode="r") as f:
        key = f.readline()

    conn = pymysql.connect(
        #host="biglab.c3svvjp5iqfn.ap-northeast-1.rds.amazonaws.com", port=3306,
        host="rds-red-w.zigbang.io", port=3306,
        user="zigbang_tableau", passwd='gksdkfma', #cryptocode.decrypt(key, AWS_SECRET),
        db="tableau", charset="utf8"
    )
    cursor = conn.cursor() 
    cursor.execute(sql_command)
    result = pd.DataFrame(cursor.fetchall())

    num_fileds = len(cursor.description)
    field_names = [i[0] for i in cursor.description]
    result.columns = field_names

    conn.close()
        
    return result

In [15]:
def load_xml_to_dataframe(response, keyword):
    import bs4 as bs
    import urllib.request
    soup = bs.BeautifulSoup(response_xml,'xml')
    
    rows = soup.find_all(keyword)
    columns = rows[0].find_all()
    
    rowList = []
    nameList = []
    columnList = []

    rowsLen = len(rows)
    columnsLen = len(columns)

    for i in range(0, rowsLen):
        columns = rows[i].find_all()

        for j in range(0, columnsLen):
            if i == 0:
                nameList.append(columns[j].name) #header
            eachColumn = columns[j].text #value
            columnList.append(eachColumn)
        rowList.append(columnList)
        columnList = []    # 다음 row의 값을 넣기 위해 비워준다

    result = pd.DataFrame(rowList, columns=nameList)
    return result

In [16]:
df = get_sql(AWS_KEY, AWS_SECRET,
        sql_command = 
        '''SELECT * FROM tableau.기업공시정보
        '''
       )

In [10]:
df.loc[df.법인등록번호 =='110111-0191398'].transpose()

,10,11,12,13,14,15,16,17,18,19,20,21,46255,46256,46257,46258
id,11,12,13,14,15,16,17,18,19,20,21,22,46256,46257,46258,46259
고유번호,00100601,00100601,00100601,00100601,00100601,00100601,00100601,00100601,00100601,00100601,00100601,00100601,00100601,00100601,00100601,00100601
기업명,(주)강원에너지,(주)강원에너지,(주)강원에너지,(주)강원에너지,(주)강원에너지,(주)강원에너지,(주)강원에너지,(주)강원에너지,(주)강원에너지,(주)강원에너지,(주)강원에너지,(주)강원에너지,(주)강원에너지,(주)강원에너지,(주)강원에너지,(주)강원에너지
종목코드,114190,114190,114190,114190,114190,114190,114190,114190,114190,114190,114190,114190,114190,114190,114190,114190
법인등록번호,110111-0191398,110111-0191398,110111-0191398,110111-0191398,110111-0191398,110111-0191398,110111-0191398,110111-0191398,110111-0191398,110111-0191398,110111-0191398,110111-0191398,110111-0191398,110111-0191398,110111-0191398,110111-0191398
사업자등록번호,124-81-46960,124-81-46960,124-81-46960,124-81-46960,124-81-46960,124-81-46960,124-81-46960,124-81-46960,124-81-46960,124-81-46960,124-81-46960,124-81-46960,124-81-46960,124-81-46960,124-81-46960,124-81-46960
기업유형,민간기업,민간기업,민간기업,민간기업,민간기업,민간기업,민간기업,민간기업,민간기업,민간기업,민간기업,민간기업,민간기업,민간기업,민간기업,민간기업
지역코드,4513014700,4513014700,4513014700,4513014700,4513014700,4513014700,4513014700,4513014700,4513014700,4513014700,4513014700,4513014700,4513014700,4513014700,4513014700,4513014700
시도,전북,전북,전북,전북,전북,전북,전북,전북,전북,전북,전북,전북,전북,전북,전북,전북
시군구,군산시,군산시,군산시,군산시,군산시,군산시,군산시,군산시,군산시,군산시,군산시,군산시,군산시,군산시,군산시,군산시


In [18]:

key = os.environ.get('companySummary_key')

url = 'http://apis.data.go.kr/1160100/service/GetCorpBasicInfoService/getCorpOutline'
url_financialStat=   'http://apis.data.go.kr/1160100/service/GetFinaStatInfoService/getSummFinaStat' 


In [20]:
data =[]# pd.DataFrame()
df_기업기본정보 = pd.DataFrame() 

for crno in df.법인등록번호.unique():
    no = crno.replace("-", "")
    try:
          params ={'serviceKey' : key, 'pageNo' : '1', 'numOfRows' : '1', 'resultType' : 'xml', 
               'basDt' : '20221001', 'crno':no}
          response_xml = requests.get(url, params=params).content
          data = load_xml_to_dataframe(response_xml, 'item')
          
          params ={'serviceKey' : key, 'pageNo' : '1', 'numOfRows' : '1', 'resultType' : 'xml', 
               'crno' : no, 'bizYear' : '2021' }
          response_xml = requests.get(url_financialStat, params=params).content
          data_financialStat = load_xml_to_dataframe(response_xml, 'item')  
          
          data = pd.concat([data, data_financialStat], axis=1)
          
          df_기업기본정보 = df_기업기본정보.append(data)
    except:
         print('찾을 수 없는 법인등록번호 : ', crno )
         pass
    
    

찾을 수 없는 법인등록번호 :  151111-0001366
찾을 수 없는 법인등록번호 :  110111-0191398
찾을 수 없는 법인등록번호 :  140111-0000246
찾을 수 없는 법인등록번호 :  110111-0370439
찾을 수 없는 법인등록번호 :  174611-0002979
찾을 수 없는 법인등록번호 :  110111-0344591
찾을 수 없는 법인등록번호 :  180111-0003391
찾을 수 없는 법인등록번호 :  124311-0008999
찾을 수 없는 법인등록번호 :  110111-0008262
찾을 수 없는 법인등록번호 :  110111-0089395
찾을 수 없는 법인등록번호 :  134711-0001159
찾을 수 없는 법인등록번호 :  120111-0003765
찾을 수 없는 법인등록번호 :  164811-0001961
찾을 수 없는 법인등록번호 :  174711-0000187
찾을 수 없는 법인등록번호 :  120111-0005703
찾을 수 없는 법인등록번호 :  110111-0150229
찾을 수 없는 법인등록번호 :  170111-0007733
찾을 수 없는 법인등록번호 :  110111-0706501
찾을 수 없는 법인등록번호 :  110111-0212889
찾을 수 없는 법인등록번호 :  110111-0052037
찾을 수 없는 법인등록번호 :  180111-0001494
찾을 수 없는 법인등록번호 :  110111-0194277
찾을 수 없는 법인등록번호 :  134411-0001840
찾을 수 없는 법인등록번호 :  110111-0149446
찾을 수 없는 법인등록번호 :  110111-0286412
찾을 수 없는 법인등록번호 :  110111-0016728
찾을 수 없는 법인등록번호 :  110111-0152134
찾을 수 없는 법인등록번호 :  110111-0175970
찾을 수 없는 법인등록번호 :  120111-0009268
찾을 수 없는 법인등록번호 :  110111-0086169
찾을 수 없는 법인

In [26]:
params ={'serviceKey' : key, 'numOfRows' : '10', 'pageNo' : '1', 'resultType' : 'xml', 
         'crno' : '1101110191398', 'bizYear' : '2019' }
response_xml = requests.get(url_financialStat, params=params).content
data_financialStat = load_xml_to_dataframe(response_xml, 'item')    
data_financialStat  

,basDt,bizYear,crno,enpBzopPft,enpCptlAmt,enpCrtmNpf,enpSaleAmt,enpTastAmt,enpTcptAmt,enpTdbtAmt,fnclDcd,fnclDcdNm,fnclDebtRto,iclsPalClcAmt
0,20191231,2019,1101110191398,-17175963153,8721837000,-35647437747,42549621578,56602062768,21124556413,35477506355,ifrs_ConsolidatedMember,연결요약재무제표,167.9443850152,-34788361860
1,20191231,2019,1101110191398,-17150531429,8721837000,-33460764640,42549621578,55106031812,22904033530,32201998282,ifrs_SeparateMember,별도요약재무제표,140.5953158417,-32601688753


In [28]:
data_financialStat.transpose()

,0,1
basDt,20191231,20191231
bizYear,2019,2019
crno,1101110191398,1101110191398
enpBzopPft,-17175963153,-17150531429
enpCptlAmt,8721837000,8721837000
enpCrtmNpf,-35647437747,-33460764640
enpSaleAmt,42549621578,42549621578
enpTastAmt,56602062768,55106031812
enpTcptAmt,21124556413,22904033530
enpTdbtAmt,35477506355,32201998282


In [37]:
df_기업기본정보.to_excel('기업기본정보.xlsx')

In [ ]:

#params ={'serviceKey' : key, 'pageNo' : '1', 'numOfRows' : '10', 'resultType' : 'xml', 
#         'basDt' : '20200509', 'crno' : '1101113892240', 'corpNm' : '메리츠자산운용' }

params ={'serviceKey' : key, 'pageNo' : '1', 'numOfRows' : '10', 'resultType' : 'xml', 
         'basDt' : '20221001', 'corpNm':'삼성전자'} #'crno':'1101117701356'}#, 'corpNm' : 'LG' }
#params ={'serviceKey' : key, 'pageNo' : '1', 'numOfRows' : '10', 'resultType' : 'xml', 
#         'basDt' : '20220701'}
response_xml = requests.get(url, params=params).content #.text.encode('utf-8')

df = load_xml_to_dataframe(response_xml, 'item')#['crno'] #.to_excel('기업개요조회.xlsx')
df.head()

In [42]:
len(df.법인등록번호.unique() )

3204

In [13]:
data = pd.read_excel('/Users/reejungkim/Documents/Git/OpenAPI_Ministry_of_land/기업기본정보.xlsx')

In [17]:
data.crno.unique()

array([1511110001366, 1101110191398, 1401110000246, ..., 1101115490472,
       1101115542702, 1101115655448])

In [ ]:
for 